In [ ]:
 # Import the dependencies
import pandas as pd
import numpy as np
import datetime

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# ORM Stuff
from sqlalchemy import create_engine, inspect, text, func

In [ ]:
df = pd.read_csv("detailed_ev_charging_stations.csv")
df.head()

In [ ]:
engine = create_engine("sqlite:///evcharging_station.sqlite")

In [ ]:
# Write to SQL (NOTE I AM USING con=engine) THIS IS WHAT WORKS APPARENTLY WHEN WRITING OUT DATA
df.to_sql(name="evcharging_station", con=engine, index=False, if_exists="append", method="multi")

In [ ]:
# Create engine using the `ev_charging_stations.sqlite` database file
#engine = create_engine("sqlite:///ev_charging_stations.sqlite")

In [ ]:
  # Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within the 'dow' table and its types
for table in tables:
    print(table)
    print("--------")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

In [ ]:
 conn = engine.connect() # Raw SQL/Pandas

In [ ]:
#sql_query = text("""
#    SELECT "Station ID", "Address", "Longitude", "Latitude"  
 #   FROM evcharging_station
  # WHERE "Address" LIKE :address_filter;""")

#address_filter = '%Bangkok%' 

#df = pd.read_sql(sql_query, con=conn, params={"address_filter": address_filter})

In [ ]:

# Import the required libraries
import hvplot.pandas
import pandas as pd

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Configure the map
map_plot_1 = df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600    
)

# Display the map plot
map_plot_1

In [ ]:
#charger type distribution
query = text("""SELECT "Charger Type", COUNT(*) AS NumberOfStations
                FROM evcharging_station
                GROUP BY "Charger Type"
                ORDER BY NumberOfStations DESC;""")

df = pd.read_sql(query, con=conn)
df.head(100)

In [ ]:
query = text("""
    SELECT
        "Latitude",
        "Longitude",
        "Charger Type",
        COUNT(*) AS NumberOfStations
    FROM
        evcharging_station 
    GROUP BY
        Latitude,
        Longitude,
        "Charger Type"
    ORDER BY
        Latitude,
        Longitude,
        "Charger Type";
""")
df = pd.read_sql(query, con=conn)
df.head(100)

In [ ]:
# 6. Create Bar Chart (using matplotlib)

# Group by Charger Type and sum the number of stations
charger_type_counts = df.groupby("Charger Type")["NumberOfStations"].sum()

plt.figure(figsize=(10, 6))  # Adjust figure size as needed
charger_type_counts.plot(kind="bar")  # Create a bar chart
plt.title("Number of Charging Stations by Charger Type")
plt.xlabel("Charger Type")
plt.ylabel("Number of Stations")
plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for readability
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()

In [ ]:
#RAW SQL
query = text("""SELECT "Installation Year", COUNT(*) AS station_count
    FROM evcharging_station
    GROUP BY "Installation Year"
    ORDER BY "Installation Year";
""")

df1 = pd.read_sql(query, con=conn)
df1.head(10)

In [ ]:
#Visualization: Line chart showing the number of charging stations installed per year.
plt.plot(df1["Installation Year"], df1["station_count"], marker="o", color="firebrick", linestyle="-")
plt.title("Charging Stations Installed Per Year")
plt.xlabel("Installation Year")
plt.ylabel("station_count")
plt.show()


In [ ]:
#average charging capacity by charger type
query = text("""SELECT "Charger Type", AVG("Charging Capacity (kW)") AS AverageChargingCapacity
                FROM evcharging_station
                GROUP BY "Charger Type"
                ORDER BY AverageChargingCapacity DESC;""")
df = pd.read_sql(query, con=conn)
df.head(10)


In [ ]:
#Average Rating by station
query = text("""SELECT "Station Operator", AVG("Reviews (Rating)") AS AverageRating
                FROM evcharging_station
                GROUP BY "Station Operator"
                ORDER BY AverageRating DESC;""")
df = pd.read_sql(query, con=conn)
df.head(10)

In [ ]:
#Visualization: Scatter plot to explore if lower charging costs lead to higher usage.

In [ ]:
#RAW SQL
query = text("""SELECT "Latitude", "Longitude"
FROM evcharging_station;
""")

df = pd.read_sql(query, con=conn)
df.head(10)

In [ ]:
#Visualization: Heatmap displaying the density of charging stations geographically.

In [ ]:
#RAW SQL
query = text("""SELECT "Renewable Energy Source", COUNT(*) AS count
FROM evcharging_station
GROUP BY "Renewable Energy Source";
""")

df = pd.read_sql(query, con=conn)
df.head(10)

In [ ]:
#Visualization: Pie chart showing the percentage of stations powered by renewable energy vs. non-renewable sources.
RenewableSource = ["yes", "no"]
count = [2566, 2434]
colors = ["green", "blue"]
explode = (0, 0.05)

plt.title("renewable vs non-renewable")
plt.pie(count, explode=explode, labels=RenewableSource, colors=colors,
        autopct="%1.1f%%", shadow=True, startangle=90)
plt.axis("equal")
plt.show()

In [ ]:
address_filter = '%Bangkok%'  # Define the filter *before* using it

query = text("""
    SELECT "Station ID" AS station_id,
           "Address" AS address,
           "Charger Type" AS charger_type  
    FROM evcharging_station
    WHERE "Address" LIKE :address_filter
    ORDER BY "Charger Type";  
""")

df = pd.read_sql(query, con=conn, params={"address_filter": address_filter}) # Pass the parameters here!

print(df.head(100))  # Print the first 10 rows

# If you want to see more rows, you can do this:
# print(df) # This will print all rows of the dataframe
# print(df.to_string()) # This will print all rows and columns of the dataframe

#conn.dispose()  # Close the connection when done

In [ ]:
# Load your dataset (assuming it's a CSV file)
df = pd.read_csv("detailed_ev_charging_stations.csv")

# Drop the 'Availability' column
df.drop(columns=['Availability'], inplace=True)

# Save the cleaned dataset
df.to_csv("cleaned_data.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# Load the data
file_path = "cleaned_data.csv"
df = pd.read_csv(file_path)
# Count the number of stations installed each year
installation_counts = df["Installation Year"].value_counts().sort_index()
# Convert to DataFrame for better readability
installation_table = pd.DataFrame({
    "Year": installation_counts.index,
    "Number of Stations Installed": installation_counts.values
})


In [ ]:
# Calculate the cumulative total of stations installed
installation_table["Total Stations at Year End"] = installation_table["Number of Stations Installed"].cumsum()
# Create a figure with two subplots
plt.figure(figsize=(12, 6))
# Plot 1: Stations Installed Each Year
plt.subplot(1, 2, 1)
plt.plot(installation_table["Year"], installation_table["Number of Stations Installed"], marker='o', linestyle='-', color='orange')
plt.xlabel("Year")
plt.ylabel("Number of Stations Installed")
plt.title("Stations Installed Each Year")
plt.grid(True)
# Plot 2: Total Stations Over the Years
plt.subplot(1, 2, 2)
plt.plot(installation_table["Year"], installation_table["Total Stations at Year End"], marker='o', linestyle='-', color='blue')
plt.xlabel("Year")
plt.ylabel("Total Stations at Year End")
plt.title("Total Stations Over the Years")
plt.grid(True)
# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
conn.close()
engine.dispose()